In [ ]:
!pip install silence_tensorflow

In [ ]:
!nvidia-smi

In [ ]:
from silence_tensorflow import silence_tensorflow
silence_tensorflow()

import tensorflow as tf
import os
from tensorflow.keras import layers

In [ ]:

print(tf.config.list_physical_devices())
strategy = tf.distribute.MirroredStrategy()

In [ ]:
SEED = 42
tf.random.set_seed(SEED)

IMG_SIZE = 224
N_FRAMES = 20
BATCH_SIZE = 4
CHANNELS = 1
AUTOTUNE = tf.data.AUTOTUNE

TRAIN_RECORD_DIR = 'Data/violence_video_train.tfrecord'
VAL_RECORD_DIR = 'Data/violence_video_val.tfrecord'

In [ ]:
def parse_tfrecord(example):
    features = {
        'label': tf.io.FixedLenFeature([], tf.int64),
        'feature': tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, features)
    video = tf.io.decode_raw(example['feature'], tf.float32)
    video = tf.reshape(video, (N_FRAMES, IMG_SIZE, IMG_SIZE, CHANNELS))
    label = tf.cast(example['label'], tf.uint8)
    return video, label

def preprocess(video, label):
    video = video / 255.0
    return video, label

In [ ]:
DATASET_SIZE = 2000

train_dataset = tf.data.TFRecordDataset(TRAIN_RECORD_DIR)
train_dataset = train_dataset.map(parse_tfrecord)
train_dataset = train_dataset.map(preprocess)

val_dataset = tf.data.TFRecordDataset(VAL_RECORD_DIR)
val_dataset = val_dataset.map(parse_tfrecord)
val_dataset = val_dataset.map(preprocess)

train_dataset = train_dataset.shuffle(buffer_size=1600)
val_dataset = val_dataset.shuffle(buffer_size=400)

train_dataset = train_dataset.batch(BATCH_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE)

train_dataset = train_dataset.prefetch(AUTOTUNE)
val_dataset = val_dataset.prefetch(AUTOTUNE)

train_dataset, val_dataset

In [ ]:
class RandomFlipVideo(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(RandomFlipVideo, self).__init__()

    def call(self, inputs):
        if tf.random.uniform(()) > 0.5:
            return tf.map_fn(lambda x: tf.image.flip_left_right(x), inputs)
        return inputs

class RandomRotationVideo(tf.keras.layers.Layer):
    def __init__(self, max_rotation=0.3, **kwargs):
        super(RandomRotationVideo, self).__init__()
        self.max_rotation = max_rotation

    def call(self, inputs):
        return tf.map_fn(self.rotate, inputs)

    def rotate(self, video):
        random_factor = self.max_rotation * self.max_rotation * 2 - self.max_rotation
        return tfa.image.rotate(video, random_factor)

    def get_config(self):
        config = super().get_config().copy()
        return config


In [ ]:
def create_model():

    NEURONS = 16
    DROPOUT = 0.5
    N_LAYERS = 1

    model = tf.keras.models.Sequential()

    model.add(layers.InputLayer(input_shape=(N_FRAMES, IMG_SIZE, IMG_SIZE, CHANNELS)))
    model.add(RandomFlipVideo())
    model.add(RandomRotationVideo(0.3))

    model.add(layers.ConvLSTM2D(
        filters=8, 
        kernel_size=3,
        padding='same'))
    model.add(layers.TimeDistributed(layers.Dropout(0.5)))

    model.add(layers.Flatten())

    model.add(layers.Dense(128))
    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(
        loss='binary_crossentropy',
        optimizer=tf.keras.optimizers.Nadam(),
        metrics=['accuracy'],
    )

    return model

In [ ]:
!pip install tensorflow_addons

import matplotlib.pyplot as plt
import numpy as np
import tensorflow_addons as tfa
%matplotlib inline

In [ ]:
fig = plt.figure(figsize=(30, 7))
rows=2
columns=10

for batch_video, batch_label in val_dataset:
    for video, label in zip(batch_video, batch_label):
    print(label)

    factor = tf.random.uniform(()) * 0.35 * 2 - 0.35
    new_vid = tfa.image.rotate(video, factor)

    print(video.shape)
    print(new_vid.shape)

    for i, frame in enumerate(new_vid):
        img = tf.cast(frame * 255, np.uint8).numpy()
        img = np.reshape(img, (224, 224))
        fig.add_subplot(rows, columns, i+1)
        plt.imshow(img, 'gray')
        plt.axis('off')
        plt.title(label.numpy())

        break
    break

In [ ]:
early_stopper = tf.keras.callbacks.EarlyStopping('val_accuracy', patience=20, restore_best_weights=True)
reduce_lr_on_plataeu = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=5)

from datetime import datetime
time_date = datetime.now().strftime("%I-%M-%p")

check_point = tf.keras.callbacks.ModelCheckpoint(f'Checkpoints/violence_model_{time_date}.h5', save_best_only=True)

with strategy.scope():
    model = create_model()

history = model.fit(train_dataset, 
                    validation_data=val_dataset, 
                    epochs=5, 
                    callbacks=[check_point, early_stopper, reduce_lr_on_plataeu, tf.keras.callbacks.TensorBoard("tb_logs")], 
                    use_multiprocessing=True, 
                    workers=16,
                    batch_size=BATCH_SIZE,
                    )

model.save(f'PersonDetection_temp.h5')

In [ ]:
print(history.history)

In [ ]:
metrics = model.evaluate(val_dataset)
metrics

In [ ]:
model.save(f'Models/Violence_Acc_{metrics[1]}.h5')

In [ ]:
model = tf.keras.models.load_model('/content/PersonDetection_temp.h5', custom_objects= {
    'RandomFlipVideo': RandomFlipVideo,
    'RandomRotationVideo': RandomRotationVideo,
})

In [ ]:
model.evaluate(val_dataset)

In [ ]:
for batch_vids, batch_labels in val_dataset:
    for vid, lab in zip(batch_vids, batch_labels):
        print(vid)
        print(lab)
        video = vid
        label = lab
        break
    break

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
fig = plt.figure(figsize=(35, 7))
rows = 2
cols = 10
for i, frame in enumerate(vid):
    fig.add_subplot(rows, cols, i+1)
    plt.imshow(np.reshape(frame, (224, 224)), cmap='gray')
    plt.axis('off')
    plt.title(lab.numpy())

print(model.predict(tf.expand_dims(vid, 0))[0][0])